## Mathematical Principles in Pattern Recognition (2016/2017)
$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\bI}{\mathbf{I}}$
$\newcommand{\bA}{\mathbf{A}}$
$\newcommand{\bQ}{\mathbf{Q}}$
$\newcommand{\bR}{\mathbf{R}}$
$\newcommand{\bX}{\mathbf{X}}$
$\newcommand{\bsigma}{\boldsymbol{\sigma}}$
$\newcommand{\bmu}{\boldsymbol{\mu}}$
$\newcommand{\bpi}{\boldsymbol{\pi}}$

In [ ]:
%pylab inline

In this tutorial we will work on one final project, instead of the step-by-step exercises of previous labs.

## 1. Final project: Overfitting
**[100 points]** Create a project about overfitting in the remainder of this notebook using markdown cells for equations and comments and code cells for code. Make sure to touch upon the following topics:
1. Use the wine data set to show what *overfitting* is in terms of a regression problem. (see: white_data.npy, white_targets.npy, red_data.npy, red_targets.npy)
2. Discuss how low and high *bias* and *variance* come into play here using figure(s), and write down what *model complexity* has to do with it.
3. One way to deal with your overfitted data in a frequentist setting is regularized regression. Use your pick of regularized regression here and apply a cross-validation scheme to determine the regularization parameter $\lambda$. 
4. Finally, shortly explain the Bayesian point-of-view on what you have done and how this would prevent overfitting. How could you use the Bayesian method to select the best model for your data? Contrast between model averaging and model selection and use the latter to select a good model.

For more background information, refer to Bishop 1.1, 1.3, 1.5, 3.1.4, 3.2, 3.4!
 
Notes on implementation:
* Make sure that your hand-in is self-contained, understandable to read from start to end with an introduction about overfitting and overall conclusion or outlook.
* This time we emphasize code cleanness and will allocate **[20 points]** to the readability of your code and graphical output.
* Use your own implementations instead of standard Python machine-learning tools, like `sk-learn`. More standard modules like `numpy` are allowed as always.
* As always: make sure you submit all included data and files necessary to run your notebook out-of-the-box!

# Overfitting: Drunk On Regularized Regression
### Lab 5
Micha de Groot, 10434410  
Jan Schutte, 11030844


## Introduction
In this lab we aim to give insight into the problem of overfitting in regression models.
Recall that minimizing the Sum of Squared Errors will yield the best fit for a given dataset:

$$ \sum_{i = 1}^n (y_i - f(w, x_i))^2 $$

Where function $f$ gives us the prediction for input $x$ and weight vector $w$.

This problem with this method is that we create a fit that exactly fits our dataset, if we then use our weight vector on another set of data we will see that the model predicts it poorly. This happens because this method creates a very complicated model that will go through almost all of the points of it's dataset, thus not taking into account the random noise we presume is on our data. In this lab we show how regularization might solve this problem by assigning a penalty to complex models.

### Dataset
The dataset used here descibes the quality of a red and white wine given 11 properties of that wine (fixed acidity,  volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, 
pH, sulphates, alcohol) with a target value of the quality score between 0 and 10.


### Helper functions
Below is the boilerplate code for reading the dataset, creating the designmatrix and splitting the datasets into two.

In [ ]:
def read_input_data():
    """ Read inputdata files in Python 3 format."""
    white_feat = load('white_data_beter.npy')
    white_tar = load('white_targets_beter.npy')
    red_feat = load('red_data_beter.npy')
    red_tar = load('red_targets_beter.npy')
    return white_feat, white_tar, red_feat, red_tar


def design_matrix(n, white_feat, red_feat):
    """ Creates design matrix for white and red wine datasets.
    n : degree of polynomials
    white_feat: Input features for white dataset
    red_feat: Input features for red dataset
    """
    white_size = shape(white_feat)
    red_size = shape(red_feat)
    feat_size = 1 + n*white_size[1]
    X = ones((white_size[0]+red_size[0], feat_size))
    if n==1:
        X[:white_size[0], 1:] = white_feat
        X[white_size[0]:, 1:] = red_feat
        return X
    elif n== 2:
        X[:white_size[0], 1:white_size[1]+1] = white_feat
        X[white_size[0]:, 1:white_size[1]+1] = red_feat
        X[:white_size[0], white_size[1]+1:] = square(white_feat)
        X[white_size[0]:, white_size[1]+1:] = square(red_feat)
        return X
    else:
        return X
    
def train_test_split(X, target, ratio):
    """ Shuffle and split dataset into test set and training set."""
    assert len(X) == len(target)
    p = numpy.random.permutation(len(target))
    X, target = X[p], target[p]
    
    X_train = X[:int(floor(len(X)*ratio)),:]
    X_test = X[int(floor(len(X)*ratio)):,:]
    
    target_train = target[:int(floor(len(target)*ratio)),:]
    target_test = target[int(floor(len(target)*ratio)):,:]
    
    return X_train, X_test, target_train, target_test

### Gradient descent
This is an iterative method for finding minimal value of the derivative of a function. Here we use it to find the lowest value of our cost function. The algorithm starts at some point in the parameter space, and it calculates the derivative of that point for each dimension of w. This will give the direction of the slope in that point, which will be the direction where we will choose the next point. By moving in the direction of the slope we will eventually fall into the 'valley' where the cost function is minimized. 

The alpha parameter is the learning rate, a scalar value that tells us how far away we choose the next point. Choosing this parameter is important, as a too small value will not yield results fast enough and too value that is too big will cause the algorithm to overshoot the 'valley' we are looking for.

### Regularization
Both the gradient descent and cost functions have a regularization parameter lamdba, this parameter inflates the values of the cost function by adding the square of the weight vector to the cost thus favoring a weight vector that is small in size.

In [ ]:
def gradient_descent(alpha, n, X, y, labda):
    w = ones((shape(X)[1], 1))
    for i in range(n):
        w -= alpha * ((X.T@(X@w-y))/shape(y)[0] - 2*labda*w)
    return w

def cost(X, w, y, labda):
    temp = X@w-y
    temp = (temp.T@temp) / shape(X)[0] - labda*sqrt((w.T@w))
    return temp[0,0]**2

### Evaluation of test results
The wine dataset has been used to train several models with linear regression. We can see in the test results below that the design matrix that contains the second degree polynomials don't give a useful model since the regression does not converge untill the learning rate is lower than 1e-9. In comparison, the linear polyomials converges to a decent result when the learning rate is 1e4 times higher. This shows that using second degree polynomials causes the linear regession model to overfit.


In [ ]:
labda = 0.001 # magic number
alpha = 0.0001 # magic numer
iterations = 100000
n = 2


w_f, w_t, r_f, r_t = read_input_data()

for j in range(5):
    alpha *= 0.1
    for i in range(1,n+1):
        X = design_matrix(i, w_f, r_f)
        target = append(w_t, r_t, axis=0)
        X_train, X_test, target_train, target_test = train_test_split(X, target, 0.90)

        w = gradient_descent(alpha, iterations, X_train, target_train, labda)
        wine_cost = cost(X_test, w, target_test, labda)
        print("The cost of the test set is %s."%(wine_cost))
        print("There are polynomials of degree %d in the design matrix and the learning rate is %s.\n"%(i,alpha) )

### Choosing the lambda parameter
The regularization parameter we choose affects the penalty we assign to a more complex model. This parameter balances the model's complexity; we don't want a model that is too simple, but also don't want it too complicated as it will cause our model to overfit. What we have done below is train our model using the training set and calculating the cost on the test set while increasing the lambda from zero to one. 

As can be seen from the graph below, a low regularization parameter causes the model to underfit and results in a high cost on the test set. Then when the regularization parameter increases even more the cost again increases as we are overfitting our model to the trainset. We see for this model the cost is minimized for lambda $0.6$.

In [ ]:
# What happens to the cost when we increase the regularization parameter?
X = design_matrix(1, w_f, r_f)
target = append(w_t, r_t, axis=0)
X_train, X_test, target_train, target_test = train_test_split(X, target, 0.90)
alpha = 0.0001
iterations = 1000
c = []
labda_space = linspace(0, 1.2, 100)
for l in labda_space:
    w = gradient_descent(alpha, iterations, X_train, target_train, l)
    c.append(cost(X_test, w, target_test, l))
c = array(c)

plt.plot(labda_space, c);
plt.ylabel("cost")
plt.xlabel("lambda")

# Print minimum lambda
idx = np.argwhere(c == np.min(c))[0,0]
found_lambda = labda_space[idx]
print("Best lambda:%f"% found_lambda)

### Plotting model predictions
In the graphs below we have plotted the prediction of our model for the inputs of the test set. Each graph displays one of the features' dimension. In blue are the actual test set data and in orange our predictions. 

In [ ]:
def plot_result(X, w, y, dimension=1):
    plt.scatter(X[:, dimension], y, alpha=0.3)
    plt.scatter(X[:, dimension], X@w, alpha=0.3)

w = gradient_descent(alpha, iterations, X_train, target_train, found_lambda)
    
# Plot testdata with found weights for each dimension
plt.figure(figsize=(15,20))
for dim in range(12):
    plt.subplot(6, 2, dim + 1)
    plot_result(X_test, w, target_test, dimension=dim)
plt.show()


### Bayesian approach

The problem of overfitting can also be solved by using a bayesian approach, by assigning a prior distribution to the parameter space the model becomes more robust. The posterior distribution is now dependent on both the prior  and likelihood, as opposed to the frequentist approach where we only look at the likelihood. This also has the added benefit that small dataset can still result in good predictions given that a good prior distribution is chosen.